In [1]:
from tsai.basics import *
import json
import numpy as np
import fastai
import fastai.tabular
from fastai.vision.all import *
from CNN import *
import matplotlib.pyplot as plt
from mocatml.utils import *
from mocatml.data import *

In [2]:
with open("data_config.json", "r") as outfile:
    data = json.load(outfile, strict=False)
extent = list(data['extent'])
data_numpy = np.load('data.npy')

d, n_epochs, window_size, batch_size = 256, 35, 60, 4096
num_sim = 100
model = AutoEncoder(d)
load_model(f'model/d_{d}_epoch_{n_epochs}.pkl', model, None)

In [3]:
with open(f'model/indices_d_{d}_epoch_{n_epochs}.txt', "r") as f:
    indices = f.read().split()
    
split = float(indices[0])
indices = [int(i) for i in indices[1:]]

train_indices = indices[:int(split * num_sim)]
val_indices = indices[int(split * num_sim):]

In [4]:
data_enc = []
for i in range(num_sim):
    encoded = model.encrypt(torch.tensor(data_numpy[i], dtype=torch.float32).unsqueeze(0).transpose(0, 1))
    data_enc.append(encoded.detach().numpy())
data_enc = np.array(data_enc)
data_enc.shape

(100, 2436, 256)

In [5]:
sh = data_enc.shape
data_enc_reshaped = data_enc.reshape((sh[0], sh[-1], sh[1]))
X_train, y_train = apply_sliding_window_3d(data_enc_reshaped[train_indices], 
                                           window_len=10,
                                           stride=1,
                                           seq_first=False,
                                           horizon=1)
X_valid, y_valid = apply_sliding_window_3d(data_enc_reshaped[val_indices], 
                                           window_len=10,
                                           stride=1,
                                           seq_first=False,
                                           horizon=1)
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((230470, 256, 10), (230470, 256, 1), (12130, 256, 10), (12130, 256, 1))

In [6]:
X, y, splits = combine_split_data([X_train, X_valid], [y_train, y_valid])
X.shape, y.shape, splits

((242600, 256, 10),
 (242600, 256, 1),
 ((#230470) [0,1,2,3,4,5,6,7,8,9...],
  (#12130) [230470,230471,230472,230473,230474,230475,230476,230477,230478,230479...]))

In [23]:
import torch
learn = None
gc.collect()
torch.cuda.empty_cache()

In [24]:
model_name = "PatchTST"
learn = TSForecaster(X, y, splits, bs=2048, arch=model_name)
lr = learn.lr_find().valley

OutOfMemoryError: CUDA out of memory. Tried to allocate 1024.00 MiB (GPU 0; 31.74 GiB total capacity; 30.03 GiB already allocated; 612.88 MiB free; 30.17 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
def get_n_params(model):
    pp=0
    for p in list(model.parameters()):
        nn=1
        for s in list(p.size()):
            nn = nn*s
        pp += nn
    return pp

get_n_params(learn)

In [ ]:
epochs = 60
learn.fit_one_cycle(epochs, lr, cbs=[ShowGraphCallback()])